# BMG Labtech CLARIOstar Plus

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance (implemented), Fluorescence (planned), Luminescence (planned)</li> <li><b>Optical Systems:</b> Dual LVF Monochromator, Physical Filters, UV/Vis Spectrometer (220–1000 nm combined range)</li> <li><b>Plate Delivery:</b> Drawer</li> <li><b>Additional Features:</b> Temperature control, Shaking, Rapid full-plate autofocus, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

**CLARIOstar Plus vs CLARIOstar:** The CLARIOstar Plus (post-2019) replaced the original CLARIOstar. Both share the dual LVF Monochromator + filter + spectrometer architecture. The Plus adds rapid full-plate autofocus, newer PMT options (e.g. far-red), and Voyager control software. EDR (Enhanced Dynamic Range) was introduced after 2024 and is **not** present on all CLARIOstar Plus units. This backend targets the CLARIOstar Plus but may also work with the original CLARIOstar (untested).

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar Plus, create a `PlateReader` frontend
instance that uses a `CLARIOstarPlusBackend` as its backend.

Currently supported: `setup()`, `open()`, `close()`, `stop()`, status polling,
device identification (EEPROM, firmware, usage counters), temperature control,
and absorbance measurement (discrete wavelengths).
Fluorescence and luminescence will be added in later phases.

In [1]:
import logging
import os
import time

from pylabrobot import verbose

verbose(True)

# Write a DEBUG-level trace log so every sent/received frame is captured.
# Uses .txt extension because .log is in .gitignore.
# If the existing log is older than 60 s it belongs to a previous run — start
# a fresh file.  Otherwise append so re-running this cell mid-session doesn't
# discard data.
_log_path = "clariostar_trace.txt"
_mode = "a"
if os.path.exists(_log_path):
    age = time.time() - os.path.getmtime(_log_path)
    if age > 60:
        _mode = "w"
else:
    _mode = "w"

_plr_logger = logging.getLogger("pylabrobot")
_plr_logger.setLevel(logging.DEBUG)
_fh = logging.FileHandler(_log_path, mode=_mode)
_fh.setLevel(logging.DEBUG)
_fh.setFormatter(logging.Formatter("%(asctime)s %(levelname)s %(message)s"))
_plr_logger.addHandler(_fh)
print(f"Trace log: {_log_path} ({'new file' if _mode == 'w' else 'appending'})")

Trace log: clariostar_trace.txt (new file)


In [2]:
from pylabrobot.plate_reading import PlateReader
from pylabrobot.plate_reading.bmg_labtech import CLARIOstarPlusBackend

clariostar_plus_backend = CLARIOstarPlusBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_plus_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0,
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_plus_backend = CLARIOstarPlusBackend(device_id="FT1234AB")
```

In [3]:
await pr.setup()

2026-02-26 11:21:24,986 - pylabrobot.io.ftdi - INFO - Successfully opened FTDI device: 430-2621
2026-02-26 11:21:25,246 - pylabrobot - INFO - read 24 bytes: 0200180c011506260000030000000000ea00f1e00003260d
2026-02-26 11:21:25,246 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-26 11:21:25,283 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:25,284 - pylabrobot - INFO - status: {'standby': False, 'busy': False, 'running': False, 'valid': True, 'unread_data': False, 'lid_open': False, 'initialized': True, 'reading_wells': False, 'z_probed': True, 'plate_detected': True, 'drawer_open': False, 'filter_cover_open': False, 'temperature_bottom': 23.4, 'temperature_top': 24.1}
2026-02-26 11:21:25,321 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:25,359 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration,
initialize command). The backend polls status until the device is no longer busy.
```

---
### Defining & Assigning a Plate

Every measurement requires a plate to be assigned to the plate reader (to tell the firmware what the positions for reading are).

In [4]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Usage: Drawer

The CLARIOstar loads and unloads plates via a motorized drawer.

In [19]:
# Open the drawer (plate out)
await pr.open()

2026-02-26 11:22:48,604 - pylabrobot - INFO - read 24 bytes: 0200180c012506260000030000000000f200f9e00003460d
2026-02-26 11:22:48,607 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-26 11:22:48,645 - pylabrobot - INFO - read 24 bytes: 0200180c012506260000030000000000f200f9e00003460d
2026-02-26 11:22:48,651 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'valid': True, 'unread_data': False, 'lid_open': False, 'initialized': True, 'reading_wells': False, 'z_probed': True, 'plate_detected': True, 'drawer_open': False, 'filter_cover_open': False, 'temperature_bottom': 24.2, 'temperature_top': 24.9}
2026-02-26 11:22:48,790 - pylabrobot - INFO - read 24 bytes: 0200180c012506260000030000000000f200f9e00003460d
2026-02-26 11:22:48,792 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'valid': True, 'unread_data': False, 'lid_open': False, 'initialized': True, 'reading_wel

In [20]:
# Place your plate on the drawer (manually or via a robotic arm), then close it (plate in)
await pr.close()

2026-02-26 11:22:52,861 - pylabrobot - INFO - read 24 bytes: 0200180c012506210000030000000000f200f9e00003410d
2026-02-26 11:22:52,866 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-26 11:22:52,909 - pylabrobot - INFO - read 24 bytes: 0200180c012506210000030000000000f200f9e00003410d
2026-02-26 11:22:52,911 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'valid': True, 'unread_data': False, 'lid_open': False, 'initialized': True, 'reading_wells': False, 'z_probed': False, 'plate_detected': False, 'drawer_open': True, 'filter_cover_open': False, 'temperature_bottom': 24.2, 'temperature_top': 24.9}
2026-02-26 11:22:53,050 - pylabrobot - INFO - read 24 bytes: 0200180c012506210000030000000000f200f9e00003410d
2026-02-26 11:22:53,051 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'valid': True, 'unread_data': False, 'lid_open': False, 'initialized': True, 'reading_we

---
## Device Identification

EEPROM configuration, firmware info, usage counters, and status queries.

In [7]:
config = clariostar_plus_backend.configuration
for key, value in config.items():
    print(f"  {key:25s} {value}")

print()
modes = await clariostar_plus_backend.request_available_detection_modes()
print(f"  Available detection modes: {', '.join(modes) if modes else 'none'}")

2026-02-26 11:21:35,458 - pylabrobot - INFO - read 271 bytes: 02010f0c070506260000000100000a0101010100000100ee0200000f00e2030000000000000304000001000001020000000000000000000032000000000000000000000000000000000000000074006f0000000000000065000000dc050000000000000000f4010803a70408076009da08ac0d0000000000000000000000000000000000000000000000000100000001010000000000000001010000000000000012029806ae013d0a4605ee01fbff700c00000000a40058ff8e03f20460ff5511fe0b55118f1a170298065aff970668042603bc14b804080791009001463228460a0046071e00200398062003f2062103d40628002c01900146001e00001411001209ac0d60090000000000220a0d
2026-02-26 11:21:35,460 - pylabrobot - INFO - EEPROM: 263 bytes, head=070506260000000100000a0101010100


  serial_number             430-2621
  firmware_version          1.35
  firmware_build_timestamp  Nov 20 2020 11:51:21
  model_name                CLARIOstar Plus
  machine_type_code         1574
  max_temperature           45.0
  has_absorbance            True
  has_fluorescence          True
  has_luminescence          True
  has_alpha_technology      True
  excitation_monochromator_max_nm 750
  emission_monochromator_max_nm 994
  excitation_filter_slots   4
  dichroic_filter_slots     3
  emission_filter_slots     4

  Available detection modes: absorbance, absorbance_spectrum, fluorescence, luminescence, alpha_technology


In [8]:
counters = await clariostar_plus_backend.request_usage_counters()
for name, value in counters.items():
    print(f"  {name:25s} {value:>12,}")

2026-02-26 11:21:35,517 - pylabrobot - INFO - read 50 bytes: 0200320c210506260000001e06680000078e000004e9000003db00027714000012fa0000000a0000000a0000000a0005350d


  flashes                      1,967,720
  testruns                         1,934
  wells                          125,700
  well_movements                  98,700
  active_time_s                  161,556
  shake_time_s                     4,858
  pump1_usage                         10
  pump2_usage                         10
  alpha_time                          10


In [9]:
status = await clariostar_plus_backend.request_machine_status()
for flag, value in status.items():
    print(f"  {flag:20s} {value}")

2026-02-26 11:21:35,572 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d


  standby              False
  busy                 False
  running              False
  valid                True
  unread_data          False
  lid_open             False
  initialized          True
  reading_wells        False
  z_probed             True
  plate_detected       True
  drawer_open          False
  filter_cover_open    False
  temperature_bottom   23.4
  temperature_top      24.1


In [10]:
if await clariostar_plus_backend.sense_plate_present():
    print("Plate is in the drawer")
else:
    print("No plate detected")

if await clariostar_plus_backend.is_ready():
    print("Machine is ready")
else:
    print("Machine is busy")

2026-02-26 11:21:36,597 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:21:36,597 - pylabrobot - INFO - read 17 bytes: 0200180c0105062600000000000000000d
2026-02-26 11:21:36,599 - pylabrobot - WARNING - status request: bad frame on attempt 1/5 (Size field says 24, got 17 bytes)
2026-02-26 11:21:36,637 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,675 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d


Plate is in the drawer
Machine is ready


---
## Temperature

The CLARIOstar Plus has a dual-plate incubator (bottom + top heating elements) with
0.1 °C resolution. The firmware exposes three temperature modes via command family `0x06`:

| Mode | Payload | Effect |
|------|---------|--------|
| **OFF** | `0x0000` | Disable heating and sensor readout |
| **MONITOR** | `0x0001` | Activate sensors only (no heating) |
| **SET** | target x 10 | Heat to target °C and activate sensors |

```{important}
The firmware treats all three as a single-state register: each new `0x06` command
**replaces** the previous one. `measure_temperature()` is safe to call while heating
is active -- it internally activates monitoring in an idempotent way (checks if sensors
are already reporting and returns immediately if so), so it will never overwrite an
active heating setpoint.
```

### Reading the current temperature

`measure_temperature()` returns the current plate temperature. On the first call it
activates the sensors internally (~200 ms warmup); subsequent calls detect that sensors
are already reporting and return instantly.

In [11]:
temp = await clariostar_plus_backend.measure_temperature()
print(f"Temperature: {temp:.1f} °C")

target = clariostar_plus_backend.get_target_temperature()
print(f"Target temperature: {target}")  # None -- no heating active

2026-02-26 11:21:36,717 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,755 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d


Temperature: 23.4 °C
Target temperature: None


In [12]:
# Read each sensor individually, or the mean of both
bottom = await clariostar_plus_backend.measure_temperature(sensor="bottom")
top = await clariostar_plus_backend.measure_temperature(sensor="top")
mean = await clariostar_plus_backend.measure_temperature(sensor="mean")

print(f"Bottom plate: {bottom:.1f} °C")
print(f"Top plate:    {top:.1f} °C")
print(f"Mean:         {mean:.1f} °C")

2026-02-26 11:21:36,803 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,842 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,880 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,918 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,958 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:36,996 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d


Bottom plate: 23.4 °C
Top plate:    24.1 °C
Mean:         23.8 °C


### Checking temperature state

The CLARIOstar Plus firmware does not echo back the heating setpoint in its
status response, so pylabrobot tracks the target temperature on the host side.
`get_target_temperature()` returns the setpoint in °C, or `None` if no heating
is active. Because this is host-managed state (not queried from the device),
it uses `get_` nomenclature rather than `request_`.

In [13]:
target = clariostar_plus_backend.get_target_temperature()
print(f"Target temperature: {target}")  # None -- sensors active but no heating setpoint

Target temperature: None


### Heating to a target temperature

`start_temperature_control(target)` heats the incubator to the target °C.
Use `measure_temperature()` to monitor progress -- it is safe to call during
active heating (see note above).

`stop_temperature_control()` stops heating but keeps sensors active so
`measure_temperature()` continues to work without re-activating them.

In [14]:
import asyncio

# Before heating: no target set
print(f"Target temperature: {clariostar_plus_backend.get_target_temperature()}")  # None

# Start heating to 37 °C
await clariostar_plus_backend.start_temperature_control(37.0)
print(f"\nAfter start_temperature_control(37.0):")
print(f"Target temperature: {clariostar_plus_backend.get_target_temperature()}")  # 37.0

# Monitor until we reach the setpoint (or close enough)
for _ in range(5):
    temp = await clariostar_plus_backend.measure_temperature()
    print(f"Bottom plate: {temp:.1f} °C")
    if temp >= 36.5:
        print("Reached target range!")
        break
    await asyncio.sleep(2)

# Stop heating but keep sensors active
await clariostar_plus_backend.stop_temperature_control()
print(f"\nAfter stop_temperature_control():")
print(f"Target temperature: {clariostar_plus_backend.get_target_temperature()}")  # None

# Sensors still work after stopping heating
await asyncio.sleep(1)
temp = await clariostar_plus_backend.measure_temperature()
print(f"Bottom plate: {temp:.1f} °C")

2026-02-26 11:21:37,065 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:37,103 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:37,141 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000000000e00001380d


Target temperature: None


2026-02-26 11:21:37,479 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d
2026-02-26 11:21:37,517 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ea00f1e00003130d



After start_temperature_control(37.0):
Target temperature: 37.0
Bottom plate: 23.4 °C


2026-02-26 11:21:39,555 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000e900f1e00003120d
2026-02-26 11:21:39,593 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000e900f1e00003120d


...

Bottom plate: 23.6 °C


2026-02-26 11:21:47,867 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000000000e00001380d



After stop_temperature_control():
Target temperature: None


2026-02-26 11:21:49,208 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ef00f9e00003200d
2026-02-26 11:21:49,246 - pylabrobot - INFO - read 24 bytes: 0200180c010506260000000000000000ef00f9e00003200d


Bottom plate: 23.9 °C


---
## Absorbance

Single-wavelength and multi-wavelength discrete absorbance measurements with configurable
optics, scan direction, and shaking.

### Single wavelength

In [15]:
results = await pr.read_absorbance(
    wavelength=600,
    use_new_return_type=True,
)

print(f"OD at 600nm, well A1: {results[0]['data'][0][0]}")

2026-02-26 11:21:49,350 - pylabrobot - INFO - read 53 bytes: 0200350c032504260000000004bc0000018c010000003000000001010000000000000002000000260001000000020000ca00030a0d
2026-02-26 11:21:50,355 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:21:50,355 - pylabrobot - WARNING - data poll: bad frame (Response too short (0 bytes)), retrying
2026-02-26 11:21:50,421 - pylabrobot - INFO - read 24 bytes: 0200180c013504260000d50000000000f000f9e00004240d
2026-02-26 11:21:50,422 - pylabrobot - INFO - measurement progress: 0/0
2026-02-26 11:21:51,424 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:21:51,426 - pylabrobot - WARNING - status request: bad frame on attempt 1/5 (Response too short (0 bytes))
2026-02-26 11:21:51,481 - pylabrobot - INFO - read 24 bytes: 0200180c012504260000040100000400f100fae000034a0d
2026-02-26 11:21:52,483 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:21:52,484 - pylabrobot - WARNING - data poll: bad frame (Response 

OD at 600nm, well A1: 0.0804477048717826


In [17]:
import pandas as pd

pd.DataFrame(results[0]["data"])

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.080448,0.084968,0.088013,0.087822,0.086796,0.086771,0.094269,0.089785,0.102217,0.087046,0.089581,0.098745
1,0.216780,0.087292,0.090914,0.089777,0.091104,0.088368,0.156621,0.096315,0.093795,0.087799,0.088331,0.087028
2,0.375260,0.087150,0.088045,0.088709,0.086687,0.087388,1.152732,0.112453,0.087482,0.095222,0.090525,0.090523
3,0.648215,0.087818,0.088046,0.087201,0.086648,0.081574,0.100180,0.093521,0.092978,0.104795,0.094636,0.107976
4,1.076914,0.090378,0.082517,0.088236,0.087822,0.079491,1.254172,0.100029,0.091902,0.150064,0.088270,0.106797
5,1.615380,0.088510,0.087097,0.089024,0.085843,0.105787,0.086958,0.094974,0.118773,0.088744,0.122659,0.152436
6,2.009436,0.090450,0.086855,0.088674,0.086758,0.088945,1.049017,0.093695,0.095993,0.086528,0.094514,0.086735
7,1.418570,0.086275,0.088818,0.092641,0.086817,0.081443,0.084692,0.087532,0.084713,0.093444,0.086084,0.087633


### Output format: `report`

`read_absorbance` supports three output formats via the `report` parameter:

| `report=` | Output | Description |
|-----------|--------|-------------|
| `"optical_density"` (default) | OD values | `OD = -log10(T)` where `T = (sample / c_hi) × (r_hi / ref)` |
| `"transmittance"` | Percent transmittance | `T% = T × 100` |
| `"raw"` | Raw detector counts | Unprocessed counts + calibration metadata |

The raw mode is useful for debugging, custom calibration, or when you need access to the
underlying detector values and reference channel data.

In [21]:
# Optical density (default)
od_results = await pr.read_absorbance(wavelength=600, report="optical_density",
                                      use_new_return_type=True)
print(f"OD:            {od_results[0]['data'][0][0]:.4f}")

# Percent transmittance
trans_results = await pr.read_absorbance(wavelength=600, report="transmittance",
                                         use_new_return_type=True)
print(f"Transmittance: {trans_results[0]['data'][0][0]:.2f} %")

# Raw detector counts (includes calibration metadata)
raw_results = await pr.read_absorbance(wavelength=600, report="raw",
                                       use_new_return_type=True)
print(f"Raw counts:    {raw_results[0]['data'][0][0]:.0f}")
print(f"Chromatic cal: {raw_results[0]['chromatic_cal']}")
print(f"Reference cal: {raw_results[0]['reference_cal']}")

2026-02-26 11:23:35,484 - pylabrobot - INFO - read 53 bytes: 0200350c032504260000000004bc0000018c010000003000000001010000000000000002000000260001000000020000ca00030a0d
2026-02-26 11:23:36,486 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:23:36,488 - pylabrobot - WARNING - data poll: bad frame (Response too short (0 bytes)), retrying
2026-02-26 11:23:36,554 - pylabrobot - INFO - read 24 bytes: 0200180c013504260000d50000000000f100f9e00004250d
2026-02-26 11:23:36,554 - pylabrobot - INFO - measurement progress: 0/0
2026-02-26 11:23:37,559 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:23:37,562 - pylabrobot - WARNING - status request: bad frame on attempt 1/5 (Response too short (0 bytes))
2026-02-26 11:23:37,630 - pylabrobot - INFO - read 24 bytes: 0200180c012504260000040100000400f100f9e00003490d
2026-02-26 11:23:38,633 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:23:38,634 - pylabrobot - WARNING - data poll: bad frame (Response 

OD:            0.0806


2026-02-26 11:24:17,324 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:24:17,325 - pylabrobot - WARNING - data poll: bad frame (Response too short (0 bytes)), retrying
2026-02-26 11:24:17,390 - pylabrobot - INFO - read 24 bytes: 0200180c013504260000d50000000000f100f9e00004250d
2026-02-26 11:24:17,391 - pylabrobot - INFO - measurement progress: 0/0
2026-02-26 11:24:18,392 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:24:18,393 - pylabrobot - WARNING - status request: bad frame on attempt 1/5 (Response too short (0 bytes))
2026-02-26 11:24:18,444 - pylabrobot - INFO - read 24 bytes: 0200180c012504260000040100000400f100f8e00003480d
2026-02-26 11:24:19,446 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:24:19,447 - pylabrobot - WARNING - data poll: bad frame (Response too short (0 bytes)), retrying
2026-02-26 11:24:19,486 - pylabrobot - INFO - read 24 bytes: 0200180c013504260000d50000000000f100f8e00004240d
2026-02-26 11:24:19,487 - p

Transmittance: 82.87 %


2026-02-26 11:24:58,120 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:24:58,121 - pylabrobot - WARNING - data poll: bad frame (Response too short (0 bytes)), retrying
2026-02-26 11:24:58,176 - pylabrobot - INFO - read 24 bytes: 0200180c013504260000d50000000000f000f8e00004230d
2026-02-26 11:24:58,177 - pylabrobot - INFO - measurement progress: 0/0
2026-02-26 11:24:59,180 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:24:59,181 - pylabrobot - WARNING - status request: bad frame on attempt 1/5 (Response too short (0 bytes))
2026-02-26 11:24:59,244 - pylabrobot - INFO - read 24 bytes: 0200180c012504260000040100000400f000f8e00003470d
2026-02-26 11:25:00,246 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:25:00,247 - pylabrobot - WARNING - data poll: bad frame (Response too short (0 bytes)), retrying
2026-02-26 11:25:00,286 - pylabrobot - INFO - read 24 bytes: 0200180c013504260000d50000000000f000f8e00004230d
2026-02-26 11:25:00,287 - p

Raw counts:    6437489
Chromatic cal: (7760347, 68701)
Reference cal: (36169, 0)


### Multi-wavelength

Pass a list of up to 8 discrete wavelengths. The firmware measures all wavelengths in a
single plate pass — each well is illuminated once per wavelength before the optic head
moves to the next well. Results are returned as a list of dicts, one per wavelength.

Each wavelength uses its own calibration pair for the OD calculation, so accuracy is
maintained across the full wavelength range.

In [22]:
results = await pr.read_absorbance(
    wavelength=260,
    wavelengths=[260, 280, 450, 600, 750],
    use_new_return_type=True,
)

for r in results:
    print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]:.4f}")

2026-02-26 11:25:38,996 - pylabrobot - WARNING - timed out reading response
2026-02-26 11:25:38,998 - pylabrobot - INFO - read 18 bytes: 0200350c032504260000000002610000370d


FrameError: Size field says 53, got 18 bytes

### Partial well selection

In [ ]:
column_1_wells = [plate.get_item(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True,
)

### Optics: flashes and well scan mode

`flashes` controls how many flashes the light source fires per well (higher = lower noise,
slower measurement). The allowed range depends on `well_scan` mode:

| `well_scan=` | Flashes | Description | Useful for |
|---|---|---|---|
| `"point"` (default) | 1–200 | Single measurement at well centre | Fast screening |
| `"orbital"` | 1–44 | Average of points on a circular path | Reduce meniscus effects |
| `"spiral"` | 1–127 | Average of points on a spiral path | Dense/turbid samples |
| `"matrix"` | 1–200 | Grid of points (2×2 to 30×30) | Not yet implemented |

`scan_diameter_mm` sets the scan circle/spiral diameter (1–6 mm, ignored for `"point"`).


In [ ]:
# Orbital scan with 7 flashes and 3 mm diameter
results = await pr.read_absorbance(
    wavelength=600,
    flashes=7,
    well_scan="orbital",
    scan_diameter_mm=3,
    use_new_return_type=True,
)

print(f"OD at 600nm (orbital), well A1: {results[0]['data'][0][0]:.4f}")

### Scan direction

Three parameters control the order in which the optic head visits wells:

| Parameter | Values | Default | Effect |
|---|---|---|---|
| `vertical` | `True` / `False` | `True` | `True` = column-major (A1→H1→A2…), `False` = row-major (A1→A12→B1…) |
| `unidirectional` | `True` / `False` | `True` | `True` = same direction each pass, `False` = serpentine (bidirectional) |
| `corner` | `"TL"` `"TR"` `"BL"` `"BR"` | `"TL"` | Starting corner of the scan path |

The defaults (`vertical=True`, `unidirectional=True`, `corner="TL"`) match the CLARIOstar
factory preset and are appropriate for most assays. Changing the direction or starting corner
can be useful for kinetic measurements where read order matters.

In [ ]:
# Horizontal serpentine scan starting from bottom-right
results = await pr.read_absorbance(
    wavelength=600,
    vertical=False,
    unidirectional=False,
    corner="BR",
    use_new_return_type=True,
)

### Shaking

The CLARIOstar can shake the plate before reading to resuspend settled particles or
mix reagents. When `shake_mode` is set, all three shake parameters are **required**
to prevent silent misconfiguration (e.g. forgetting `shake_speed_rpm` and getting no shake):

| Parameter | Type | Description |
|---|---|---|
| `shake_mode` | `str` or `None` | `None` = no shake, `"orbital"`, `"double_orbital"`, or `"linear"` |
| `shake_speed_rpm` | `int` | Shake speed in RPM (multiples of 100, 100-700). Required when `shake_mode` is set. |
| `shake_duration_s` | `int` | Duration of shaking in seconds (> 0). Required when `shake_mode` is set. |
| `settling_time_s` | `float` | Wait time after shaking before reading (0.0-1.0 s). Required when `shake_mode` is set. |

```{important}
When `shake_mode` is `None` (default), the shake parameters must also be `None`.
When `shake_mode` is set, you must explicitly provide `shake_speed_rpm`,
`shake_duration_s`, and `settling_time_s` — there are no implicit defaults.
```


In [ ]:
# Orbital shake at 300 RPM for 5 seconds, no settling delay before reading
results = await pr.read_absorbance(
    wavelength=600,
    well_scan="orbital",
    scan_diameter_mm=3,
    flashes=7,
    shake_mode="orbital",
    shake_speed_rpm=300,
    shake_duration_s=5,
    settling_time_s=0,
    use_new_return_type=True,
)

### Non-blocking measurement (`wait=False`)

By default, `read_absorbance` polls the device incrementally until the measurement
is complete (`wait=True`). Setting `wait=False` fires the measurement and returns
an empty list immediately, letting you retrieve the data later with
`request_absorbance_results()`.

**Why two modes?**

| | Blocking (`wait=True`) | Non-blocking (`wait=False`) |
|---|---|---|
| **Control flow** | Awaits until data is ready | Returns immediately |
| **Best for** | Simple scripts, sequential workflows | Multi-instrument orchestration |
| **Throughput** | Plate reader sits idle between calls | Overlap measurement with other work |

In a typical lab automation workflow the plate reader measurement takes 10-30 seconds.
With blocking mode, the entire program waits. With non-blocking mode, that time can be
used to drive a liquid handler, move plates, or start measurements on other instruments
-- the same pattern as `asyncio` tasks but at the instrument level. This is especially
valuable in high-throughput screening where plate reader time is often the bottleneck.


In [ ]:
# Fire-and-forget: start measurement, return immediately (empty list)
await pr.read_absorbance(wavelength=600, wait=False, use_new_return_type=True)

# ... do other work while the plate reader is measuring ...

# Check status until the device is no longer busy
status = await clariostar_plus_backend.request_machine_status()
while status["busy"]:
    print(f"Measuring... busy={status['busy']}")
    await asyncio.sleep(1)
    status = await clariostar_plus_backend.request_machine_status()

# Retrieve and parse the completed measurement (backend returns list of dicts directly)
results = await clariostar_plus_backend.request_absorbance_results(
    plate, plate.get_all_items(), [600]
)

print(f"OD at 600nm, well A1: {results[0]['data'][0][0]:.4f}")


---
### Measurement Timing Reference

Empirical timing data measured from USB packet captures on a CLARIOstar Plus
(firmware v1.35) with a 96-well Corning plate. All measurements
include ~4.5 s fixed overhead (drawer close + plate positioning).

#### Effect of well scan mode (96 wells, 600 nm)

| Scan mode | Flashes | Data phase | Total time | ms/well |
|-----------|---------|------------|------------|---------|
| Point     | 1       | 16.3 s     | ~21 s      | 170     |
| Point     | 5       | 31.4 s     | ~36 s      | 327     |
| Point     | 20      | 34.6 s     | ~39 s      | 360     |
| Orbital 3 mm | 5    | 56.7 s     | ~62 s      | 591     |
| Orbital 3 mm | 7    | 67.5 s     | ~73 s      | 703     |
| Orbital 5 mm | 7    | 79.6 s     | ~85 s      | 829     |
| Spiral 3 mm  | 15   | 82.1 s     | ~87 s      | 855     |
| Spiral 4 mm  | 15   | 115.7 s    | ~121 s     | 1205    |

**Key insight — flash marginal cost is tiny:** going from 1 to 5 flashes nearly
doubles the time, but 5 to 20 flashes barely changes it (31.4 s to 34.6 s). The
instrument fires all flashes in rapid succession at each well position, so most
of the time is spent on mechanical travel between wells, not flashing.

#### Effect of wavelength count (96 wells, point scan, 1 flash)

| Wavelengths | Data phase | ms/well |
|-------------|------------|---------|
| 1 (450 nm)            | 16.25 s | 169 |
| 2 (450 + 600 nm)      | 16.22 s | 169 |
| 3 (450 + 600 + 660 nm)| 16.27 s | 169 |

**Multiple wavelengths are free:** all wavelengths are measured simultaneously in
a single pass. The data phase is identical regardless of wavelength count.

#### Effect of well count (point scan, 5 flashes)

| Wells | Data phase | ms/well |
|-------|------------|---------|
| 1     | 0.8 s      | 792     |
| 8     | 2.9 s      | 362     |
| 48    | 15.8 s     | 329     |
| 96    | 31.4 s     | 327     |

The ms/well rate converges at ~327 ms for large well counts. The high per-well
cost at low counts reflects fixed overhead within the measurement cycle.

#### Effect of traversal order (96 wells, point scan, 1 flash)

| Direction | Data phase | ms/well |
|-----------|------------|---------|
| Horizontal serpentine   | 16.0 s | 167 |
| Bidirectional vertical  | 16.2–17.3 s | 169–180 |
| Unidirectional vertical | 21.1 s | 220 |

**Unidirectional is ~30% slower** than serpentine because the optic head must
return to the start of each column instead of reversing direction.

#### Effect of shaking (96 wells, orbital 3 mm, 7 flashes)

| Configuration | Shake phase | Data phase | Total |
|---------------|-------------|------------|-------|
| No shake      | —           | 67.5 s     | 72.6 s |
| Orbital 300 rpm / 5 s  | 7.3 s | 65.1 s | 78.4 s |
| Orbital 500 rpm / 5 s  | 7.3 s | 64.0 s | 77.3 s |
| Orbital 300 rpm / 10 s | 12.1 s | 63.5 s | 82.6 s |
| Linear 300 rpm / 5 s   | 7.4 s | 63.9 s | 77.3 s |

Shaking adds a clean pre-measurement phase (~2.3 s ramp overhead beyond the
configured duration) but does not affect the measurement itself.

```{note}
**Manual spec comparison:** The operating manual (0430B0006B, p.4) states
read times of 8 s (96-well), 15 s (384-well), and 28 s (1536-well) at 1 flash.
Our measured data phase for 96 wells at 1 flash is ~16.3 s — roughly double the
spec. The manual likely quotes the raw flash time excluding data transfer overhead,
or refers to an optimised firmware mode. All timings here include full USB data
retrieval as experienced by the calling program.
```

---
## Hardware Validation Tests

Systematic tests for every absorbance parameter. Run these cells on a physical
CLARIOstar Plus with a plate loaded to verify correct firmware communication.
Each cell prints a PASS/FAIL summary based on return shape and value sanity.

In [ ]:
import traceback, time

def _v(results, label, *, n_wl=1, n_wells=96, expect_cal=False):
    """Validate read_absorbance results: shape, types, optional cal keys."""
    try:
        assert isinstance(results, list), f"expected list, got {type(results)}"
        assert len(results) == n_wl, f"expected {n_wl} dicts, got {len(results)}"
        for r in results:
            assert "data" in r and "wavelength" in r, f"missing keys: {list(r.keys())}"
            flat = [v for row in r["data"] for v in row]
            assert len(flat) == n_wells, f"expected {n_wells} values, got {len(flat)}"
            for v in flat:
                assert isinstance(v, (int, float)), f"non-numeric: {v}"
            if expect_cal:
                assert "chromatic_cal" in r and "reference_cal" in r
        print(f"  PASS  {label}")
    except Exception:
        print(f"  FAIL  {label}")
        traceback.print_exc()

async def _abs(label, n_wl=1, n_wells=96, expect_cal=False, **kw):
    """Run read_absorbance with given kwargs, validate, return results."""
    kw.setdefault("wavelength", 600)
    kw["use_new_return_type"] = True
    _plr_logger.info("--- %s ---", label)
    t0 = time.monotonic()
    r = await pr.read_absorbance(**kw)
    dt = time.monotonic() - t0
    _v(r, f"{label} ({dt:.1f}s)", n_wl=n_wl, n_wells=n_wells, expect_cal=expect_cal)
    return r

In [ ]:
### T1: Well scan modes — point vs orbital vs spiral, varying diameters
print("=== T1: Well scan modes ===")
await _abs("T1a point",         well_scan="point")
await _abs("T1b orbital 3mm",   well_scan="orbital", scan_diameter_mm=3, flashes=7)
await _abs("T1c orbital 5mm",   well_scan="orbital", scan_diameter_mm=5, flashes=7)
await _abs("T1d spiral 3mm",    well_scan="spiral",  scan_diameter_mm=3, flashes=15)
await _abs("T1e spiral 4mm",    well_scan="spiral",  scan_diameter_mm=4, flashes=15)

In [ ]:
### T2: Flash counts — extremes
print("=== T2: Flash counts ===")
await _abs("T2a 1 flash",   flashes=1)
await _abs("T2b 5 flashes", flashes=5)
await _abs("T2c 20 flashes", flashes=20)

In [ ]:
### T3: Scan direction — all 4 corners, vertical/horizontal, uni/bidi
print("=== T3: Scan direction ===")
await _abs("T3a TL vert uni (default)", corner="TL", vertical=True,  unidirectional=True)
await _abs("T3b TR vert uni",           corner="TR", vertical=True,  unidirectional=True)
await _abs("T3c BL vert uni",           corner="BL", vertical=True,  unidirectional=True)
await _abs("T3d BR vert uni",           corner="BR", vertical=True,  unidirectional=True)
await _abs("T3e TL horiz bidi",         corner="TL", vertical=False, unidirectional=False)
await _abs("T3f TL vert bidi",          corner="TL", vertical=True,  unidirectional=False)
await _abs("T3g BR horiz uni",          corner="BR", vertical=False, unidirectional=True)

In [ ]:
### T4: Wavelengths — single, multi, UV, max count
print("=== T4: Wavelengths ===")
await _abs("T4a 450nm single",    wavelength=450)
await _abs("T4b 260nm UV",        wavelength=260)
await _abs("T4c dual 450+600",    wavelength=450, wavelengths=[450, 600], n_wl=2)
await _abs("T4d triple 450+600+660", wavelength=450, wavelengths=[450, 600, 660], n_wl=3)
await _abs("T4e 5-wavelength",    wavelength=260, wavelengths=[260, 280, 450, 600, 750], n_wl=5)
await _abs("T4f 8-wavelength (max)", wavelength=260,
           wavelengths=[260, 280, 350, 450, 530, 600, 700, 750], n_wl=8)

In [ ]:
### T5: Partial well selection — single well, column, rows, scattered
print("=== T5: Partial wells ===")
await _abs("T5a single well A1",  wells=[plate.get_item("A1")], n_wells=1)
await _abs("T5b column 1 (8w)",   wells=[plate.get_item(f"{r}1") for r in "ABCDEFGH"], n_wells=8)
await _abs("T5c rows A-D (48w)",  wells=plate.get_items("A1:D12"), n_wells=48)
await _abs("T5d scattered 3w",    wells=[plate.get_item("A1"), plate.get_item("D6"),
                                         plate.get_item("H12")], n_wells=3)

In [ ]:
### T6: Shake modes — orbital, linear, double_orbital; speed and duration
print("=== T6: Shake modes ===")
_sk = dict(well_scan="orbital", scan_diameter_mm=3, flashes=7)
await _abs("T6a orbital 300rpm 5s",        **_sk, shake_mode="orbital",        shake_speed_rpm=300, shake_duration_s=5,  settling_time_s=0.1)
await _abs("T6b orbital 500rpm 5s",        **_sk, shake_mode="orbital",        shake_speed_rpm=500, shake_duration_s=5,  settling_time_s=0.1)
await _abs("T6c orbital 300rpm 10s",       **_sk, shake_mode="orbital",        shake_speed_rpm=300, shake_duration_s=10, settling_time_s=0.1)
await _abs("T6d linear 300rpm 5s",         **_sk, shake_mode="linear",         shake_speed_rpm=300, shake_duration_s=5,  settling_time_s=0.1)
await _abs("T6e double_orbital 300rpm 5s", **_sk, shake_mode="double_orbital", shake_speed_rpm=300, shake_duration_s=5,  settling_time_s=0.1)

In [ ]:
### T7: Settling time
print("=== T7: Settling time ===")
_st = dict(well_scan="orbital", scan_diameter_mm=3, flashes=7,
           shake_mode="orbital", shake_speed_rpm=300, shake_duration_s=5)
await _abs("T7a settle 0s",   **_st, settling_time_s=0)
await _abs("T7b settle 0.1s", **_st, settling_time_s=0.1)
await _abs("T7c settle 0.5s", **_st, settling_time_s=0.5)

In [ ]:
### T8: Report modes — OD, transmittance, raw
print("=== T8: Report modes ===")
await _abs("T8a optical_density", report="optical_density")
await _abs("T8b transmittance",   report="transmittance")
await _abs("T8c raw",             report="raw", expect_cal=True)

In [ ]:
### T9: Combinations — cross-feature interactions
print("=== T9: Combinations ===")
await _abs("T9a orbital + dual wl",  wavelength=450, wavelengths=[450, 600], n_wl=2,
           well_scan="orbital", scan_diameter_mm=3, flashes=7)
await _abs("T9b shake + dual wl",    wavelength=450, wavelengths=[450, 600], n_wl=2,
           well_scan="orbital", scan_diameter_mm=3, flashes=7,
           shake_mode="orbital", shake_speed_rpm=300, shake_duration_s=5, settling_time_s=0.1)
await _abs("T9c shake + partial",    wells=[plate.get_item(f"{r}1") for r in "ABCDEFGH"], n_wells=8,
           well_scan="orbital", scan_diameter_mm=3, flashes=7,
           shake_mode="orbital", shake_speed_rpm=300, shake_duration_s=5, settling_time_s=0.1)
await _abs("T9d spiral + dual wl",   wavelength=450, wavelengths=[450, 600], n_wl=2,
           well_scan="spiral", scan_diameter_mm=3, flashes=15)
await _abs("T9e orbital + single A1", wells=[plate.get_item("A1")], n_wells=1,
           well_scan="orbital", scan_diameter_mm=3, flashes=7)
await _abs("T9f dual wl + partial",  wavelength=450, wavelengths=[450, 600], n_wl=2,
           wells=[plate.get_item(f"{r}1") for r in "ABCDEFGH"], n_wells=8)
await _abs("T9g dual wl + settle",   wavelength=450, wavelengths=[450, 600], n_wl=2,
           well_scan="orbital", scan_diameter_mm=3, flashes=7,
           shake_mode="orbital", shake_speed_rpm=300, shake_duration_s=3, settling_time_s=0.5)

In [ ]:
### T10: Non-blocking round-trip
print("=== T10: Non-blocking ===")
await pr.read_absorbance(wavelength=600, wait=False, use_new_return_type=True)
status = await clariostar_plus_backend.request_machine_status()
while status["busy"]:
    await asyncio.sleep(1)
    status = await clariostar_plus_backend.request_machine_status()
results = await clariostar_plus_backend.request_absorbance_results(
    plate, plate.get_all_items(), [600])
_v(results, "T10a non-blocking round-trip")
print("\n=== All hardware validation tests complete ===")

---
## Planned Features

The sections below cover measurement modes that are not yet implemented.
Code cells are commented out and will be uncommented as each feature lands.

### Fluorescence

Basic, custom gain/bandwidth, bottom-optic.

In [20]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
# )
#
# print(f"GFP fluorescence, well A1: {results[0]['data'][0][0]}")

In [ ]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
#     flashes=50,
# )

In [ ]:
# Bottom-optic fluorescence (e.g. for cell-based assays)
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=4.5,
# )

### Luminescence

Basic luminescence, partial well selection.

In [23]:
# results = await pr.read_luminescence(
#     focal_height=13.0,
# )
#
# print(f"Temperature: {results[0]['temperature']:.1f} °C")
# print(f"Well A1 RLU: {results[0]['data'][0][0]}")

In [ ]:
# selected_wells = [plate.get_item("A1"), plate.get_item("D6"), plate.get_item("H12")]
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     wells=selected_wells,
# )

In [ ]:
# row_A = [plate.get_item(f"A{col}") for col in range(1, 13)]
#
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=6.0,
#     wells=row_A,
#     shake_mode="orbital",
#     shake_speed_rpm=300,
#     shake_duration_s=3,
#     settling_time_s=0.1,
#     corner="BR",
# )

---
## Reference: How Absorbance Calculation Works

The machine does not measure absorbance directly. It measures **raw detector counts**
and the backend converts them to transmittance and then to OD.

**OD formula:**

```
T = (sample / c_hi) × (r_hi / ref)
OD = -log10(T)
```

Where:
- `sample` = per-well sample detector count
- `ref` = per-well reference detector count
- `c_hi` = chromatic calibration value for this wavelength
- `r_hi` = reference calibration value

**Multi-wavelength data layout:**

The firmware response contains data as groups of per-well u32 values followed by
calibration pairs. The number of groups scales with wavelength count:

| Wavelengths | Groups | Calibration pairs | Layout |
|:-----------:|:------:|:-----------------:|--------|
| 1 | 4 | 4 | WL1, chrom2, chrom3, reference |
| 2 | 5 | 5 | WL1, WL2, chrom2, chrom3, reference |
| 3 | 6 | 6 | WL1, WL2, WL3, chrom2, chrom3, reference |
| W | W+3 | W+3 | WL1…WLW, chrom2, chrom3, reference |

Each wavelength's OD is computed using **its own calibration pair** (WL1 → cal\[0\],
WL2 → cal\[1\], etc.) and the shared reference detector (last group, last cal pair).
The backend detects the number of groups dynamically from the payload size, so it
handles any wavelength count without configuration.

See `DESIGN.md` in `pylabrobot/plate_reading/bmg_labtech/` for the full protocol reference.

---
## Closing Connection

In [29]:
pr.unassign_child_resource(plate)

In [30]:
await pr.stop()

2026-02-24 14:58:36,931 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000fd0103e00002310d
2026-02-24 14:58:36,936 - pylabrobot - WARNING - _stop_temperature_monitoring sends OFF -- this also disables heating
2026-02-24 14:58:36,978 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 14:58:37,016 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 14:58:37,054 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d


This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.